In [1]:
import torch
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.nn import Embedding

In [3]:
torch.empty(3, dtype=torch.long).random_(5)

tensor([0, 0, 4])

In [4]:
torch.randn(3, 5, requires_grad=True)

tensor([[ 0.5569,  0.0115,  0.7086,  1.2292,  0.3242],
        [-0.0392,  0.4016,  2.5511, -1.7723, -1.5389],
        [-0.8195,  0.5356,  0.2913, -0.3434,  0.3000]], requires_grad=True)

In [3]:
d = dict({"a": 1, "b": 2})

In [4]:
len(d)

2

In [29]:
import pandas as pd
df = pd.DataFrame({"cat_1": [1, 2, 3], "g": ['a', 'a', 'b']})

In [31]:
grouped_df = df.groupby("g")

In [43]:
grouped_df.get_group("a").to_dict(orient='list')

{'cat_1': [1, 2], 'g': ['a', 'a']}

In [45]:
import polars as pl

In [49]:
pdf = pl.DataFrame({"cat_1": [1, 2, 3], "g": ['a', 'a', 'b']})

In [53]:
gpdf =  pdf.group_by("g")

In [ ]:
gpdf.to

TypeError: 'GroupBy' object is not subscriptable

In [2]:
semples = [
    (torch.tensor([1, 2]), 0, '0'),
    (torch.tensor([3, 4, 7]), 1, '1'), 
    ]

In [3]:
batch = {
   'seq': [],
   'target': [],
}
for semple in semples:
   seq, t, _ = semple
   batch['seq'].append(seq)
   batch['target'].append(t)


In [4]:
batch

{'seq': [tensor([1, 2]), tensor([3, 4, 7])], 'target': [0, 1]}

In [5]:
batch['seq']

[tensor([1, 2]), tensor([3, 4, 7])]

In [6]:
padded_seq = pad_sequence(batch['seq'], batch_first=True, padding_value=0) 

In [9]:
# B*S
padded_seq

tensor([[1, 2, 0],
        [3, 4, 7]])

In [8]:
padded_seq.shape

torch.Size([2, 3])

In [10]:
e_layer1 = Embedding(num_embeddings=8, embedding_dim=3, padding_idx=0)
e_layer2 = Embedding(num_embeddings=8, embedding_dim=2, padding_idx=0)

In [11]:
e1_t = e_layer1(padded_seq)
e2_t = e_layer2(padded_seq)

In [14]:
# B(2)*L(3)*D(2)
e2_t

tensor([[[ 2.1258, -0.2839],
         [ 1.6081, -1.0939],
         [ 0.0000,  0.0000]],

        [[ 1.5444,  1.4519],
         [ 1.1300,  0.9714],
         [ 0.1448,  0.5127]]], grad_fn=<EmbeddingBackward0>)

In [16]:
e2_t.shape

torch.Size([2, 3, 2])

In [17]:
f_t = torch.ones(*list(e2_t.shape))

In [18]:
f_t

tensor([[[1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.]]])

In [19]:
f_t.shape

torch.Size([2, 3, 2])

In [20]:
e1_t

tensor([[[ 0.7141, -0.3449, -0.8548],
         [ 0.8378, -0.3786, -2.0781],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.1777,  2.2153, -0.6737],
         [-0.4224, -0.3284,  0.3313],
         [ 0.6919,  1.1233,  1.6236]]], grad_fn=<EmbeddingBackward0>)

In [33]:
e1_t[:, :, -1]

tensor([[-0.2298,  1.0594,  0.0000],
        [-0.0022, -1.5224,  0.7959]], grad_fn=<SelectBackward0>)

In [ ]:
# B*SL*ED(1)
e1_t[:, :, -1].unsqueeze(2)

tensor([[[-0.2298],
         [ 1.0594],
         [ 0.0000]],

        [[-0.0022],
         [-1.5224],
         [ 0.7959]]], grad_fn=<UnsqueezeBackward0>)

In [28]:
# B, SL, D_1(3) + D_2(2) + D_3(2) 
inputs = torch.cat([e1_t, e2_t, f_t], dim=2)

In [27]:
inputs

tensor([[[ 0.7141, -0.3449, -0.8548,  2.1258, -0.2839,  1.0000,  1.0000],
         [ 0.8378, -0.3786, -2.0781,  1.6081, -1.0939,  1.0000,  1.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000]],

        [[ 0.1777,  2.2153, -0.6737,  1.5444,  1.4519,  1.0000,  1.0000],
         [-0.4224, -0.3284,  0.3313,  1.1300,  0.9714,  1.0000,  1.0000],
         [ 0.6919,  1.1233,  1.6236,  0.1448,  0.5127,  1.0000,  1.0000]]],
       grad_fn=<CatBackward0>)

In [25]:
inputs.shape

torch.Size([2, 3, 7])

In [58]:
torch.tensor(e1_t.shape[1])

tensor(3)

In [72]:
lengths = torch.tensor([2, 3]).cpu()

In [73]:
packed_input = pack_padded_sequence(e1_t, lengths=lengths, batch_first=True, enforce_sorted=False)

In [67]:
e1_t

tensor([[[ 0.5936, -1.8150, -0.2298],
         [ 0.2582, -0.3254,  1.0594],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 1.1526, -2.1812, -0.0022],
         [ 1.0306, -0.5221, -1.5224],
         [ 0.7259,  0.1155,  0.7959]]], grad_fn=<EmbeddingBackward0>)

In [74]:
packed_input

PackedSequence(data=tensor([[ 1.1526, -2.1812, -0.0022],
        [ 0.5936, -1.8150, -0.2298],
        [ 1.0306, -0.5221, -1.5224],
        [ 0.2582, -0.3254,  1.0594],
        [ 0.7259,  0.1155,  0.7959]], grad_fn=<PackPaddedSequenceBackward0>), batch_sizes=tensor([2, 2, 1]), sorted_indices=tensor([1, 0]), unsorted_indices=tensor([1, 0]))

In [75]:
pad_packed_sequence(packed_input, batch_first=True)

(tensor([[[ 0.5936, -1.8150, -0.2298],
          [ 0.2582, -0.3254,  1.0594],
          [ 0.0000,  0.0000,  0.0000]],
 
         [[ 1.1526, -2.1812, -0.0022],
          [ 1.0306, -0.5221, -1.5224],
          [ 0.7259,  0.1155,  0.7959]]], grad_fn=<IndexSelectBackward0>),
 tensor([2, 3]))

In [12]:
torch.empty(3, dtype=torch.long).random_(5)

tensor([4, 4, 1])

In [11]:
torch.randn(3, 5, requires_grad=True)

tensor([[ 1.3408,  0.1771,  0.3984,  0.5390,  0.3045],
        [ 1.3554, -1.3767,  0.2733, -0.2470, -0.0978],
        [ 0.8400, -0.1072,  1.2293, -0.2246,  1.4132]], requires_grad=True)